In [1]:
# HCP: The Hexagonal Close Packed structure

import numpy as np

from ase.build import bulk
from ase.calculators.emt import EMT
from ase.io import Trajectory, read

a0 = 3.52 / np.sqrt(2)
c0 = np.sqrt(8 / 3.0) * a0

In [2]:
traj = Trajectory('Ni.traj', 'w')

In [3]:
eps = 0.01
for a in a0 * np.linspace(1 - eps, 1 + eps, 3):
    for c in c0 * np.linspace(1 - eps, 1 + eps, 3):
        ni = bulk('Ni', 'hcp', a=a, c=c)
        ni.calc = EMT()
        ni.get_potential_energy()
        traj.write(ni)


In [5]:
from ase.build import bulk
ni = bulk('Ni', 'hcp', a=2.5, c=4.0)
ni.cell

Cell([[2.5, 0.0, 0.0], [-1.25, 2.1650635094610964, 0.0], [0.0, 0.0, 4.0]])

In [6]:
configs = read('Ni.traj@:')
energies = [config.get_potential_energy() for config in configs]
a = np.array([config.cell[0, 0] for config in configs])
c = np.array([config.cell[2, 2] for config in configs])

In [ ]:
functions = np.array([a**0, a, c, a**2, a * c, c**2])
p = np.linalg.lstsq(functions.T, energies, rcond=-1)[0]

In [9]:
p0 = p[0]
p1 = p[1:3]
p2 = np.array([(2 * p[3], p[4]),
               (p[4], 2 * p[5])])
a0, c0 = np.linalg.solve(p2.T, -p1)

with open('lattice_constant.csv', 'w') as fd:
    fd.write(f'{a0:.3f}, {c0:.3f}\n')

Using the stress tensor

In [10]:
from ase.optimize import BFGS
from ase.constraints import StrainFilter
from gpaw import GPAW, PW
ni = bulk('Ni', 'hcp', a=a0,c=c0)
calc = GPAW(mode=PW(200),xc='LDA',txt='Ni.out')
ni.calc = calc
sf = StrainFilter(ni)
opt = BFGS(sf)
opt.run(0.005)

/var/folders/hx/d75vz7pn5c3750m6x7vn09f00000gn/T/ipykernel_77647/3204051759.py:7: FutureWarning: Import StrainFilter from ase.filters
  sf = StrainFilter(ni)


      Step     Time          Energy          fmax
BFGS:    0 17:45:40      -20.918177       15.732296
BFGS:    1 17:45:42      -20.441809       21.412838
BFGS:    2 17:45:44      -21.841735        3.340633
BFGS:    3 17:45:46      -21.859873        0.853103
BFGS:    4 17:45:48      -21.854865        1.916026
BFGS:    5 17:45:51      -21.862081        0.036173
BFGS:    6 17:45:52      -21.862138        0.063478
BFGS:    7 17:45:54      -21.862079        0.025395
BFGS:    8 17:45:57      -21.862082        0.005681
BFGS:    9 17:45:58      -21.862087        0.014176
BFGS:   10 17:46:00      -21.862079        0.014349
BFGS:   11 17:46:02      -21.862078        0.018289
BFGS:   12 17:46:04      -21.862092        0.025978
BFGS:   13 17:46:06      -21.862079        0.005779
BFGS:   14 17:46:08      -21.862079        0.006992
BFGS:   15 17:46:11      -21.862079        0.018783
BFGS:   16 17:46:12      -21.862097        0.030682
BFGS:   17 17:46:15      -21.862080        0.003411


True

In [ ]:
traj = Trajectory('path.traj', 'w', ni)
opt.attach(traj)